In [ ]:
# install nlp
!pip install nlp
# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16:
    import os
    os.kill(os.getpid(), 9)

In [ ]:
import nlp

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR) #log only errors and supress warnings

## Dataset Used- Xsum



In [ ]:
dataset_train = nlp.load_dataset('xsum', split='train[0:3000]') 
dataset_valid = nlp.load_dataset('xsum', split='validation[:1000]')

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.1.0. Subsequent calls will reuse this data.


In [ ]:
print(dataset_train)
print(dataset_valid)

Dataset(schema: {'document': 'string', 'summary': 'string'}, num_rows: 3000)
Dataset(schema: {'document': 'string', 'summary': 'string'}, num_rows: 1000)


## Rouge Metric for Model Evaluation 

In [ ]:
!pip install rouge_score rouge_score
rouge_metric = nlp.load_metric("rouge") #rouge is a very standard metric for seq2seq tasks, especially summarization

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 13.0MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 
     |████████████████████████████████| 890kB 39.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=52712942d6aa850e75bee3897555c32cafe7116dedcf25f506e47d8a09157772
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

## Model to fine tune: BartForConditionalGeneration (seq2seq)

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')#bart-large is also available, but bart-base is much smaller and switching to large may or may not be worth the extra computation
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): SelfAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
 

In [ ]:
MAX_LEN = 1024
MAX_SUMMARY_LEN = 200
BATCH_SIZE = 2 #Experimented with sizes of 2, 4, 8, and 16. 2 seems to be optimal
EPOCHS = 2 #going over 4 epochs may result in overfitting
LEARNING_RATE = 5e-5 #any value from 3e-4 to 5e-5 works reasonably well
ADAM_EPS = 1e-8
NUM_BEAMS = 4

In [ ]:
params = {'batch_size': BATCH_SIZE,
          'shuffle': True,
          'num_workers': 0
        }

train_loader = DataLoader(dataset_train, **params)
valid_loader = DataLoader(dataset_valid, **params)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE, eps=ADAM_EPS)

### Train Function
Pytorch implementation

In [ ]:
def train(epoch):
  model.train()
  for _,data in enumerate(train_loader, 0):
    source = tokenizer.batch_encode_plus(data['document'], max_length=MAX_LEN, pad_to_max_length=True,return_tensors='pt', truncation='only_first')
    target = tokenizer.batch_encode_plus(data['summary'], max_length=MAX_LEN, pad_to_max_length=True,return_tensors='pt', truncation='only_first')
    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()
    #
    y = target_ids.to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous() #to create labels for target, we need to right shift the target ids
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100 #padding tokens should be marked with -100 value so the model can output correct loss
    ids = source_ids.to(device, dtype = torch.long)
    mask = source_mask.to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
    loss = outputs[0]
        
    if _%100==0:
      print(f'Epoch: {epoch}, Loss:  {loss}')
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### Validation Function

In [ ]:
def validate():
  model.eval()
  prediction = []
  reference = []
  with torch.no_grad():
      for _, data in enumerate(valid_loader, 0):
        source = tokenizer.batch_encode_plus(data['document'], truncation='only_first', max_length=MAX_LEN, pad_to_max_length=True,return_tensors='pt')
        target = tokenizer.batch_encode_plus(data['summary'], truncation='only_first', max_length=MAX_LEN, pad_to_max_length=True,return_tensors='pt')
        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        y = target_ids.to(device, dtype = torch.long)
        ids = source_ids.to(device, dtype = torch.long)
        mask = source_mask.to(device, dtype = torch.long)

        generated_ids = model.generate(
            input_ids = ids,
            attention_mask = mask,
            max_length=MAX_SUMMARY_LEN, 
            num_beams=NUM_BEAMS,
            repetition_penalty=2.5, 
            no_repeat_ngram_size=4,
            early_stopping=True
            )#can experiment with different values of num_beams and penalties
        pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
        ref = [tokenizer.decode(r, skip_special_tokens=True, clean_up_tokenization_spaces=True)for r in y]
        if(_%100 == 0):
          print(f"{_} batches complete")

        prediction.extend(pred)
        reference.extend(ref)
  return prediction, reference

### Training for 2 epochs (experimented with the number of epochs and ended up with 2

In [ ]:
EPOCHS = 2
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  5.72074556350708
Epoch: 0, Loss:  3.286314010620117
Epoch: 0, Loss:  2.2736337184906006
Epoch: 0, Loss:  4.060889720916748
Epoch: 0, Loss:  2.848393201828003
Epoch: 0, Loss:  2.937912940979004
Epoch: 0, Loss:  2.704157590866089
Epoch: 0, Loss:  2.310439348220825
Epoch: 0, Loss:  3.012713670730591
Epoch: 0, Loss:  2.742609739303589
Epoch: 0, Loss:  2.5388176441192627
Epoch: 0, Loss:  2.0063209533691406
Epoch: 0, Loss:  3.3642876148223877
Epoch: 0, Loss:  2.453787088394165
Epoch: 0, Loss:  3.0758860111236572
Epoch: 1, Loss:  1.6726274490356445
Epoch: 1, Loss:  2.2113537788391113
Epoch: 1, Loss:  2.1528077125549316
Epoch: 1, Loss:  1.5783202648162842
Epoch: 1, Loss:  2.5110907554626465
Epoch: 1, Loss:  1.91062593460083
Epoch: 1, Loss:  3.123586654663086
Epoch: 1, Loss:  2.2911932468414307
Epoch: 1, Loss:  1.6131576299667358
Epoch: 1, Loss:  1.8435837030410767
Epoch: 1, Loss:  1.945421814918518
Epoch: 1, Loss:  1.6504689455032349
Epoch: 1, Loss:  1.648804783821106
Epoch: 1

### Validate

In [ ]:
predictions, references = validate()

0 batches complete
100 batches complete
200 batches complete
300 batches complete
400 batches complete


### Evaluation

In [ ]:
score = rouge_metric.compute(predictions, references)
print(f"Rouge Score: {score}")

Rouge Score: {'rouge1': AggregateScore(low=Score(precision=0.3292641306496132, recall=0.23511238350095492, fmeasure=0.2684580570621723), mid=Score(precision=0.33831534413686704, recall=0.2417177622347908, fmeasure=0.2759174260817243), high=Score(precision=0.3474169787963121, recall=0.24936028542278277, fmeasure=0.28377873111571666)), 'rougeL': AggregateScore(low=Score(precision=0.2543264042430301, recall=0.18104704437948582, fmeasure=0.2069487307695577), mid=Score(precision=0.26338207250553236, recall=0.18740512482640626, fmeasure=0.21427045924265864), high=Score(precision=0.272857514684904, recall=0.19500692940770972, fmeasure=0.2222487740393936))}


### Some Results

In [ ]:
for i in range(10):
  print(f"Prediction: {predictions[i]}")
  print(f"Reference: {references[i]}")

Prediction:  gay and lesbian people in Uganda have been given a suspended prison sentence by the International Criminal Court.

Reference: The South African ambassador to Uganda, a former columnist for South Africa's Sunday Sun paper, has been found guilty of hate speech for an anti-gay article.

Prediction:  London Taxi Company has said it hopes to return to Saudi Arabia by the end of this year.

Reference: A Coventry-based black taxi manufacturer is to sell 400 cabs to Saudi Arabia.

Prediction:  Scottish universities are trying to make sure young people in Northern Ireland are eligible for free tuition.

Reference: A Scottish government spokesman has confirmed that sixth form pupils in the UK who hold Irish passports qualify for free university tuition in Scotland.

Prediction:  Celtic have confirmed the signing of Mikael Lustig from Swedish club Rosenborg.

Reference: Celtic have signed Sweden international right-back Mikael Lustig on a pre-contract agreement.

Prediction:  governm

In [ ]:
model.eval()
from pprint import pprint
for i in range(10):
  print(f"{i}th sample")
  art = dataset_valid[i]['document']
  summ = dataset_valid[i]['summary']
  source = tokenizer.batch_encode_plus([art], max_length=MAX_LEN, pad_to_max_length=True,return_tensors='pt', truncation=True)
  target = tokenizer.batch_encode_plus([summ], max_length=MAX_LEN, pad_to_max_length=True,return_tensors='pt', truncation=True)
  source_ids = source['input_ids']
  source_mask = source['attention_mask']
  target_ids = target['input_ids']
  target_mask = target['attention_mask']

  y = target_ids.to(device, dtype = torch.long)
  ids = source_ids.to(device, dtype = torch.long)
  mask = source_mask.to(device, dtype = torch.long)

  generated_ids = model.generate(
      input_ids = ids,
      attention_mask = mask, 
      max_length=150, 
      num_beams=4,
      repetition_penalty=2.5,
      early_stopping=True
      )
  pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
  ref = [tokenizer.decode(r, skip_special_tokens=True, clean_up_tokenization_spaces=True)for r in y]
  pprint(f"Prediction: {pred}")
  pprint(f"Reference: {ref}")

0th sample
("Prediction: [' luxury goods giant Burberry has reported a sharp rise in "
 "profits and revenue.\\n']")
("Reference: ['Luxury fashion designer Burberry has returned to profit after "
 "opening new stores and spending more on online marketing\\n']")
1th sample
('Prediction: [" six French astronauts have disappeared from Nasa\'s first '
 'Mars mission, as part of an international effort to send humans to '
 'Mars.\\n"]')
"Reference: ['There was all the pageantry of a real space launch.\\n']"
2th sample
("Prediction: [' Romanian stowaway who flew from Vienna to Heathrow Airport "
 "has been freed by police.\\n']")
("Reference: ['A 20-year-old Romanian man had a lucky escape after stowing "
 "away on a plane flying from Vienna to London.\\n']")
3th sample
("Prediction: [' who have anorexia nervosa tend to be taller than others, say "
 "researchers.\\n']")
("Reference: ['The brain naturally distorts body image - a finding which could "
 "explain eating disorders like anorexia, 